In [1]:
from pruning import *
from lw_retrain_utils import *
from evaluation import *
import json
import copy 
from datasets import load_dataset
import os
import gc
import shutil

/users/eleves-a/2021/ahmad.rammal/miniconda3/envs/colab/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "openai-community/gpt2-medium"
model, tokenizer = load_model(model_name)

In [3]:
prune_model_depth(model, tokenizer, 4)

Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors
100%|█████████▉| 280/281 [00:06<00:00, 45.33it/s]


Initial PPL: 21.97


Evaluating iterations...:   0%|          | 0/4 [00:00<?, ?it/s]
























































100%|█████████▉| 280/281 [00:05<00:00, 47.37it/s]














































































100%|█████████▉| 280/281 [00:10<00:00, 25.81it/s]







































































100%|█████████▉| 280/281 [00:08<00:00, 31.53it/s]

























































100%|█████████▉| 280/281 [00:06<00:00, 46.64it/s]

























































100%|█████████▉| 280/281 [00:06<00:00, 46.62it/s]

























































100%|█████████▉| 280/281 [00:06<00:00, 45.03it/s]






























































































100%|█████████▉| 280/281 [00:13<00:00, 20.24it/s]

























































100%|█████████▉| 280/281 [00:06<00:00, 46.34it/s]











Removed layer 2. 23 layers left. PPL: 22.45
New model size:  342226944



























































100%|█████████▉| 280/281 [00:05<00:00, 47.80it/s]








































































100%|█████████▉| 280/281 [00:09<00:00, 29.37it/s]














































































100%|█████████▉| 280/281 [00:10<00:00, 27.32it/s]

























































100%|█████████▉| 280/281 [00:05<00:00, 47.77it/s]

























































100%|█████████▉| 280/281 [00:05<00:00, 47.81it/s]




























































100%|█████████▉| 280/281 [00:07<00:00, 39.16it/s]


























































































100%|█████████▉| 280/281 [00:12<00:00, 22.07it/s]

























































100%|█████████▉| 280/281 [00:05<00:00, 47.81it/s]

























































100%|█████████▉

Removed layer 9. 22 layers left. PPL: 23.50
New model size:  329630720
























































100%|█████████▉| 280/281 [00:05<00:00, 47.58it/s]





















































100%|█████████▉| 280/281 [00:05<00:00, 49.70it/s]



















































































100%|█████████▉| 280/281 [00:11<00:00, 24.89it/s]





































































100%|█████████▉| 280/281 [00:07<00:00, 35.28it/s]























































100%|█████████▉| 280/281 [00:05<00:00, 49.51it/s]























































100%|█████████▉| 280/281 [00:05<00:00, 49.59it/s]





































































100%|█████████▉| 280/281 [00:08<00:00, 32.23it/s]



















































































100%|█████████▉| 280/281 [00:10<00:00, 26.23it/s]























































100%|█████████▉| 280/281

Removed layer 19. 21 layers left. PPL: 24.69
New model size:  317034496

















































100%|█████████▉| 280/281 [00:05<00:00, 51.62it/s]















































100%|█████████▉| 280/281 [00:05<00:00, 51.53it/s]













































































100%|█████████▉| 280/281 [00:09<00:00, 28.39it/s]







































































100%|█████████▉| 280/281 [00:08<00:00, 31.22it/s]















































100%|█████████▉| 280/281 [00:05<00:00, 51.50it/s]















































100%|█████████▉| 280/281 [00:05<00:00, 51.58it/s]































































100%|█████████▉| 280/281 [00:07<00:00, 35.86it/s]





















































































100%|█████████▉| 280/281 [00:11<00:00, 25.23it/s]















































100%|█████████▉| 280/281 [00:05<00:00, 51.56it/s]





















Removed layer 11. 20 layers left. PPL: 26.47
New model size:  304438272


In [4]:
dataset = load_dataset("stas/openwebtext-10k", trust_remote_code=True)
calibration_pass(model=model,
                 tokenizer=tokenizer,
                 dataset=dataset,
                 sample_size=128,
                 batch_size=4,)

100%|██████████| 32/32 [00:07<00:00,  4.29it/s]


In [ ]:
tokenized_dataset = tokenize_dataset(tokenizer, dataset)

os.makedirs("./saved_metrics", exist_ok=True)

training_metrics_path = "./saved_metrics/training_metrics.json"
eval_metrics_path = "./saved_metrics/eval_metrics.json"

if os.path.exists(training_metrics_path):
    with open(training_metrics_path, "r") as f:
        training_metrics = json.load(f)
else:
    training_metrics = {}

if os.path.exists(eval_metrics_path):
    with open(eval_metrics_path, "r") as f:
        eval_metrics = json.load(f)
else:
    eval_metrics = {}

num_heads = 12
mult_hidden = 3.5
embed_size = 1024
prune_model_width(model, int(mult_hidden * embed_size), num_heads, embed_size)
print(sum(t.numel() for t in model.parameters()))
param_key = f"depth=20_num_heads={num_heads}_mlp_exp={mult_hidden}_embed_size={embed_size}"
gc.collect()

if param_key+"_before_training" in eval_metrics:
    print(f"Skipping evaluation for {param_key}, already exists.")
else:
    print(f"Evaluating perplexity for {param_key} before training...")
    eval_metrics[param_key+"_before_training"] = evaluate_perplexity(model, tokenizer, stride=1024).item()

    with open(eval_metrics_path, "w") as f:
        json.dump(eval_metrics, f, indent=4)
if param_key in training_metrics:
    print(f"Skipping training for {param_key}, already exists.")
else:
    print(f"Training model for {param_key}...")

    
    trainer = trainer_gpt2(model, tokenizer, tokenized_dataset, batch_size=4, num_epochs=2, lr=2e-4) #, output_dir=f"./saved_models/{param_key}")
    trainer.train()
    training_metrics[param_key] = trainer.state.log_history

    with open(training_metrics_path, "w") as f:
        json.dump(training_metrics, f, indent=4)
    torch.cuda.empty_cache()
    gc.collect()
    
if param_key in eval_metrics:
    print(f"Skipping evaluation for {param_key}, already exists.")
else:
    print(f"Evaluating perplexity for {param_key}...")
    eval_metrics[param_key] = evaluate_perplexity(model, tokenizer, stride=1024).item()

    with open(eval_metrics_path, "w") as f:
        json.dump(eval_metrics, f, indent=4)

torch.cuda.empty_cache()
gc.collect()

262469632
Skipping evaluation for depth=20_num_heads=12_mlp_exp=3.5_embed_size=1024, already exists.
Skipping training for depth=20_num_heads=12_mlp_exp=3.5_embed_size=1024, already exists.


Step,Training Loss,Validation Loss
30,3.483300,3.120769
60,3.262500,3.037465
90,3.213400,3.007260
120,3.196100,2.990015
150,3.147400,2.985528
180,3.136900,2.980556
210,3.145200,2.979952
240,3.118700,2.979443
270,3.125000,2.979461


OSError: [Errno 122] Disk quota exceeded

In [ ]:
output_dir=f"./saved_models/{param_key}"

output_dir = f"./saved_models/{param_key}"
os.makedirs(output_dir, exist_ok=True)

torch.save(model, f"{output_dir}/model_lw_retrain.pth")
torch.save(model.state_dict(), f"{output_dir}/model_lw_retrain_state_dict.pth")

## Knowledge Distillation

In [6]:
import torch
from transformers import AdamW
from torch.nn import functional as F
from torch.utils.data import DataLoader
import knowledge_distillation as kd


# import knowledge_distillation  # Import the module
from evaluation import *

In [7]:
model_name = "openai-community/gpt2-medium"
teacher_model, tokenizer = load_model(model_name)
teacher_model.eval()
print('Models loaded successfully')

Models loaded successfully


In [9]:
dataset = load_dataset("deven367/babylm-10M-cbt", trust_remote_code=True)
train_dataset = dataset['train']
val_dataset = dataset['valid'].select(range(500))

In [10]:

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors="pt"
    )

    # Create input_ids and labels for language modeling
    input_ids = tokenized["input_ids"]
    labels = input_ids.clone()

    return {
        "input_ids": input_ids,
        "attention_mask": tokenized["attention_mask"],
        "labels": labels
    }
    
print("Tokenizing dataset...")
tokenized_train_datasets = train_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=32,  # Smaller batch size for mapping
    remove_columns=train_dataset.column_names,
    desc="Processing examples",
    load_from_cache_file=False  # Disable caching for debugging
)
tokenized_val_datasets = val_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=32,  # Smaller batch size for mapping
    remove_columns=val_dataset.column_names,
    desc="Processing examples",
    load_from_cache_file=False  # Disable caching for debugging
)
tokenized_train_datasets.set_format("torch")
tokenized_val_datasets.set_format("torch")

Tokenizing dataset...


Processing examples: 100%|██████████| 500/500 [00:00<00:00, 1395.07 examples/s]


In [11]:
steps, train_losses, val_losses = kd.train_kd(model, teacher_model, tokenized_train_datasets, tokenized_val_datasets, 
                                            batch_size=4, num_epochs=2, accumulation_steps=8, lr=5e-5, 
                                            temperature=1.0, device='cuda', log_interval=10, val_interval=250)

/users/eleves-a/2021/ahmad.rammal/miniconda3/envs/colab/lib/python3.10/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



🔄 Epoch 1/2 - Training...


Training Loss: 23.8997:   4%|▍         | 251/6500 [00:28<1:42:37,  1.01it/s]

📉 Step 250: Validation Loss = 15.3419


Training Loss: 18.5989:   8%|▊         | 502/6500 [00:57<1:11:40,  1.39it/s]

...

📉 Step 12500: Validation Loss = 10.3556


Training Loss: 9.7778:  96%|█████████▌| 6252/6500 [12:14<03:00,  1.37it/s]

📉 Step 12750: Validation Loss = 10.3608


📉 Step 13000: Validation Loss = 10.3464
✅ Epoch 2: Average Training Loss = 9.7484


📉 Epoch 2: Final Validation Loss = 10.3464


In [12]:
metrics_path = "saved_metrics/kd_metrics.json"
model_name = "width+depth"

# Load existing metrics if the file exists, otherwise start with an empty dict
if os.path.exists(metrics_path):
    with open(metrics_path, "r") as f:
        kd_metrics = json.load(f)
else:
    kd_metrics = {}

kd_metrics.setdefault(model_name, {})[model_name] = {"steps": steps, "train_losses": train_losses, "val_losses": val_losses}

# Save updated metrics back to the file
os.makedirs("saved_metrics", exist_ok=True)
with open(metrics_path, "w") as f:
    json.dump(kd_metrics, f, indent=4)

In [13]:
evaluate_perplexity(model, tokenizer, stride=1024).item()

Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors
100%|█████████▉| 280/281 [00:06<00:00, 40.95it/s]


34.3554801940918

In [14]:
test_dataset = load_dataset("ptb_text_only", split="test")
test_dataset = test_dataset.rename_columns({"sentence": "text"})
evaluate_perplexity(model, tokenizer, test=test_dataset, stride=512)

 99%|█████████▉| 205/207 [00:04<00:00, 41.07it/s]


tensor(37.9520, device='cuda:0')

In [16]:
from itertools import islice
from datasets import Dataset

bookcorpus = load_dataset("bookcorpus", split="train", streaming=True, trust_remote_code=True)
bookcorpus_test = list(islice(bookcorpus, 10_000))
bookcorpus_test_dataset = Dataset.from_list(bookcorpus_test)

evaluate_perplexity(model, tokenizer, test=bookcorpus_test_dataset, stride=128)

 99%|█████████▉| 1286/1294 [00:30<00:00, 42.37it/s]


tensor(18.7809, device='cuda:0')

In [17]:
test_dataset = load_dataset("lambada", split="test", trust_remote_code=True)
evaluate_perplexity(model, tokenizer, test=test_dataset, stride=1024)

100%|█████████▉| 419/420 [00:09<00:00, 41.94it/s]


tensor(50.3809, device='cuda:0')